In [5]:
import pandas as pd

In [6]:
path = "./logs/"

In [34]:
def trata_msg(linha):
    linha = linha.strip()
    dados = linha.split(", data=")
    data = [int(x) for x in dados[1][1:-2].split(",")]
    dados = dados[0].split(":sendBSMsg: Sender= ")
    [sender,evtId] = [int(x) for x in dados[1].split(", EvtId=")]
    date = dados[0]
    d = {"datetime":date, "sender":sender, "evtId":evtId, "seq":data[0], "temp":data[4], "pai":data[5], "dados":data}
    return d

def le_msgs(filename):
    file = open(path+filename, "r")
    lines = file.readlines()
    file.close()
    listLines = []
    for line in lines:
        d = trata_msg(line)
        listLines.append(d)
    return pd.DataFrame(listLines)

def calcula(df):
    qtdNan = df.isna().sum().sum()
    total = df.shape[0]*df.shape[1]
    return 1-qtdNan/total

def duplicados(dataframe):
    dataframe_sem_nan = dataframe.fillna(0)
    quantidade_valores_maiores_que_um = (dataframe_sem_nan > 1).sum().sum()
    return quantidade_valores_maiores_que_um

def indices_ultimas_linhas_nao_nan(dataframe):
    ultimos_indices_nao_nan = dataframe.apply(lambda coluna: coluna.last_valid_index())
    return ultimos_indices_nao_nan

def contar_nan_ate_ultimas_linhas(dataframe):
    ultimos_indices_nao_nan = indices_ultimas_linhas_nao_nan(dataframe)
    contagem_nan_por_coluna = {}
    for coluna in dataframe.columns:
        contagem_nan = dataframe[coluna].loc[:ultimos_indices_nao_nan[coluna]].isna().sum()
        contagem_nan_por_coluna[coluna] = contagem_nan
    return contagem_nan_por_coluna

def calc_qtd_tentativas(df):
    d=indices_ultimas_linhas_nao_nan(df).to_dict()
    soma=0
    for key in d:
        soma+=d[key]
    return soma

def calc_qtd_perdidos(df):
    d=contar_nan_ate_ultimas_linhas(df)
    soma=0
    for key in d:
        soma+=d[key]
    return soma

def exibe_metricas_log(filename):
    df = le_msgs(filename)
    dfpv = pd.pivot_table(df, index="seq",columns="sender",values="temp",aggfunc="count")
    qtdPerd=calc_qtd_perdidos(dfpv)
    qtdTent=calc_qtd_tentativas(dfpv)
    print("Quantidade de nós:",dfpv.shape[1])
    print("Quantidade de sequências:",dfpv.shape[0])
    print("Número da sequência máxima:",dfpv.iloc[-1].name)
    print("Quantidade de valores duplicados:",duplicados(dfpv))
    print("Quantidade de tentativas conhecidas de envio:",qtdTent)
    print("Quantidade de mensagens com certeza perdidas:",qtdPerd)
    print("Percentual de tentativas conhecidas recebidas: %.1f%%"%(100*(1-qtdPerd/qtdTent)))
    print("Percentual de mensagens recebidas:")
    print("\tTotal: %.1f%%"%(calcula(dfpv)*100))
    print("\t[-1]:  %.1f%%"%(calcula(dfpv[:-1])*100))
    print("\t[-2]:  %.1f%%"%(calcula(dfpv[:-2])*100))
    print("\t[-3]:  %.1f%%"%(calcula(dfpv[:-3])*100))

In [35]:
exibe_metricas_log("log13.txt")

Quatidade de nós: 64
Quantidado de sequências: 11
Número da sequência máxima: 10
Quantidade de valores duplicados: 3
Quantidade de tentativas conhecidas de envio: 589
Quantidade de mensagens com certeza perdidas: 39
Percentual de tentativas conhecidas recebidas: 93.4%
Percentual de mensagens recebidas:
	Total: 87.2%
	[-1]:  92.2%
	[-2]:  93.4%
	[-3]:  92.8%


In [36]:
df = le_msgs("log13.txt")
df

,datetime,sender,evtId,seq,temp,pai,dados
0,2023.11.11 22:32:22,11,1,0,500,1,"[0, 0, 0, 0, 500, 1, 0, 0, 0, 0]"
1,2023.11.11 22:32:24,12,1,0,500,11,"[0, 0, 0, 0, 500, 11, 0, 0, 0, 0]"
2,2023.11.11 22:32:24,21,1,0,500,11,"[0, 0, 0, 0, 500, 11, 0, 0, 0, 0]"
3,2023.11.11 22:32:25,22,1,0,500,11,"[0, 0, 0, 0, 500, 11, 0, 0, 0, 0]"
4,2023.11.11 22:32:29,31,1,0,500,22,"[0, 0, 0, 0, 500, 22, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
612,2023.11.11 22:42:04,72,1,9,500,61,"[9, 0, 0, 0, 500, 61, 0, 0, 0, 0]"
613,2023.11.11 22:42:05,75,1,9,500,66,"[9, 0, 0, 0, 500, 66, 0, 0, 0, 0]"
614,2023.11.11 22:42:06,67,1,9,500,66,"[9, 0, 0, 0, 500, 66, 0, 0, 0, 0]"
615,2023.11.11 22:42:06,74,1,9,500,75,"[9, 0, 0, 0, 500, 75, 0, 0, 0, 0]"


In [37]:
dfpv = pd.pivot_table(df, index="seq",columns="sender",values="temp",aggfunc="count")
dfpv

sender,11,12,13,14,15,16,17,18,21,22,...,77,78,81,82,83,84,85,86,87,88
seq,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0,...,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
